In [1]:
# =========================
# 1️⃣ IMPORTS
# =========================
import os
import zipfile
import shutil
import joblib
import numpy as np
import pandas as pd
from pprint import pprint
from datetime import datetime

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score, classification_report, confusion_matrix

# Colab download helper (works only in Colab)
try:
    from google.colab import files as colab_files
    _IN_COLAB = True
except Exception:
    _IN_COLAB = False

# for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# GPU detection (for user info only)
try:
    import torch
    gpu_available = torch.cuda.is_available()
    gpu_name = torch.cuda.get_device_name(0) if gpu_available else "No GPU"
except Exception:
    gpu_available = False
    gpu_name = "torch not installed / unavailable"

print(f"GPU available: {gpu_available}, device name: {gpu_name}")
print(f"Running at: {datetime.utcnow().isoformat()} UTC")

GPU available: True, device name: Tesla T4
Running at: 2025-10-14T16:43:39.607060 UTC


/tmp/ipython-input-2279429140.py:43: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print(f"Running at: {datetime.utcnow().isoformat()} UTC")


In [2]:
# 2️⃣ READ DATASET
# =========================
# Replace path with your Colab-mounted drive path or upload the csv to Colab working dir.
DATA_PATH = "HEAR_Dataset.csv"  # change if needed

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"{DATA_PATH} not found. Upload it to the runtime or update DATA_PATH.")

df = pd.read_csv(DATA_PATH, encoding='utf-8')
# Basic sanity check
print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())
# Expected columns: "Review Text", "Pricing", "Appointments", "Medical Staff", "Customer Service", "Emergency Services"
aspect_names = ["Pricing", "Appointments", "Medical Staff", "Customer Service", "Emergency Services"]
for a in aspect_names:
    if a not in df.columns:
        raise ValueError(f"Expected column '{a}' not found in dataset.")

# Convert Review Text to string and strip
df["Review Text"] = df["Review Text"].astype(str).str.strip()

# Optional: quick label dtype check
print("Label sample (first 5 rows):")
display(df[["Review Text"] + aspect_names].head())

# =========================
# 3️⃣ SPLIT TRAIN / TEST
# =========================
texts = df["Review Text"].tolist()
labels = df[aspect_names].values  # shape (n_samples, 5)

X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=RANDOM_STATE, stratify=None
)
print(f"Train size: {len(X_train_texts)}, Test size: {len(X_test_texts)}")

Dataset shape: (23004, 6)
Columns: ['Review Text', 'Pricing', 'Appointments', 'Medical Staff', 'Customer Service', 'Emergency Services']
Label sample (first 5 rows):


,Review Text,Pricing,Appointments,Medical Staff,Customer Service,Emergency Services
0,مستشفى كبير اشكر الدكتوره هيبات الصديقي صاحبه ...,3,3,1,1,3
1,شكرا لهم فردا فردا ….إبتداءً من الاستقبال ثم ...,3,3,1,1,3
2,شكر و تقدير لكي دكتورة ولاء عامر بالنساء و الو...,3,3,1,3,3
3,مستشفى كبير ومرتب وفيه جميع التخصصات وكفاءات ع...,1,3,1,1,3
4,سيء ثم. سيء ثم سيء ثم سيء\nمثل ماانتم شايفين ف...,0,3,0,0,3


Train size: 18403, Test size: 4601


In [3]:
# 4️⃣ TF-IDF VECTORIZATION (shared encoder)
# =========================
# Simple Arabic normalization function
import re
def normalize_arabic(text):
    text = str(text)
    # remove tashkeel
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    # normalize alef variants
    text = re.sub(r'[إأٱآا]', 'ا', text)
    # normalize ya
    text = re.sub(r'[يى]', 'ي', text)
    # normalize ta marbuta
    text = re.sub(r'ة', 'ه', text)
    # remove non-Arabic letters and digits optionally (keep space and Arabic letters and basic punctuation)
    text = re.sub(r'[^\u0600-\u06FF\s\.,!?؛،\-]', '', text)
    # collapse whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply normalization (fast)
X_train_texts = [normalize_arabic(t) for t in X_train_texts]
X_test_texts  = [normalize_arabic(t) for t in X_test_texts]

# TF-IDF vectorizer config — tuned for Arabic
vectorizer = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1,2),
    analyzer='word',
    token_pattern=r'(?u)\b\w+\b',
    lowercase=False  # we've normalized; Arabic lowercase not relevant
)

print("Fitting TF-IDF vectorizer on training texts...")
X_train_tfidf = vectorizer.fit_transform(X_train_texts)
X_test_tfidf  = vectorizer.transform(X_test_texts)
print("TF-IDF shapes:", X_train_tfidf.shape, X_test_tfidf.shape)


Fitting TF-IDF vectorizer on training texts...
TF-IDF shapes: (18403, 20000) (4601, 20000)


In [4]:
# 5️⃣ LABEL PREPARATION (no tensors)
# =========================
# Labels are already numeric (0,1,2,3). Check class distribution per aspect
for i, aspect in enumerate(aspect_names):
    unique, counts = np.unique(y_train[:, i], return_counts=True)
    print(f"{aspect} train distribution: {dict(zip(unique, counts))}")

Pricing train distribution: {np.int64(0): np.int64(1409), np.int64(1): np.int64(77), np.int64(2): np.int64(103), np.int64(3): np.int64(16814)}
Appointments train distribution: {np.int64(0): np.int64(2027), np.int64(1): np.int64(321), np.int64(2): np.int64(118), np.int64(3): np.int64(15937)}
Medical Staff train distribution: {np.int64(0): np.int64(3044), np.int64(1): np.int64(6038), np.int64(2): np.int64(432), np.int64(3): np.int64(8889)}
Customer Service train distribution: {np.int64(0): np.int64(5865), np.int64(1): np.int64(5717), np.int64(2): np.int64(154), np.int64(3): np.int64(6667)}
Emergency Services train distribution: {np.int64(0): np.int64(1265), np.int64(1): np.int64(320), np.int64(2): np.int64(33), np.int64(3): np.int64(16785)}


In [5]:
# 7️⃣ MODEL DEFINITION (MultiOutputClassifier)
# =========================
# We'll create a LogisticRegression per aspect, with class weights computed per aspect
estimators = []
from sklearn.base import clone

base_estimator = LogisticRegression(
    solver='saga',          # saga for large datasets and supports l1/l2
    multi_class='multinomial',
    max_iter=2000,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

# Compute class weights per aspect and create custom estimators with those weights
estimators_per_aspect = []
for i, aspect in enumerate(aspect_names):
    y_col = y_train[:, i]
    classes = np.unique(y_col)
    # compute_class_weight expects classes array
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_col)
    # map to dict
    cw_dict = {c: w for c, w in zip(classes, class_weights)}
    # create estimator with these weights
    est = LogisticRegression(
        solver='saga',
        multi_class='multinomial',
        max_iter=2000,
        class_weight=cw_dict,
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
    estimators_per_aspect.append(est)

# MultiOutputClassifier with LogisticRegression base; sklearn will clone the base estimator per output.
# We will pass a LogisticRegression that does not have class_weight, but then replace internal estimators manually after fit.
multi_clf = MultiOutputClassifier(base_estimator, n_jobs=-1)

In [ ]:
# 8️⃣ TRAINING SETUP
# =========================
print("Training MultiOutputClassifier (this may take a short while)...")
multi_clf.fit(X_train_tfidf, y_train)
print("Initial training done.")

# Replace each estimator with a class-weighted retrained estimator for best imbalance handling:
# Alternatively, fit separate estimators manually if you prefer more control.
print("Refitting individual estimators with computed class weights per aspect for better handling of imbalance...")
for i, est in enumerate(estimators_per_aspect):
    print(f"  Fitting estimator for aspect: {aspect_names[i]}")
    est.fit(X_train_tfidf, y_train[:, i])
    multi_clf.estimators_[i] = est
print("All aspect estimators refitted.")

Training MultiOutputClassifier (this may take a short while)...
Initial training done.
Refitting individual estimators with computed class weights per aspect for better handling of imbalance...
  Fitting estimator for aspect: Pricing


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


  Fitting estimator for aspect: Appointments


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


  Fitting estimator for aspect: Medical Staff


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


  Fitting estimator for aspect: Customer Service


In [ ]:
# 🔟 SAVING THE MODEL (vectorizer + multioutput classifier + metadata)
# =========================
save_dir = "saved_model_tfidf"
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, "multioutput_model.joblib")
vect_path  = os.path.join(save_dir, "vectorizer.joblib")
meta_path  = os.path.join(save_dir, "metadata.txt")

joblib.dump(multi_clf, model_path)
joblib.dump(vectorizer, vect_path)

with open(meta_path, "w", encoding="utf-8") as f:
    f.write("TF-IDF MultiOutput model\n")
    f.write(f"Saved at: {datetime.utcnow().isoformat()} UTC\n")
    f.write(f"Aspect names: {aspect_names}\n")
    f.write(f"Vectorizer max_features: {vectorizer.max_features}\n")

# Optionally zip the model folder for download
zip_filename = "saved_model_tfidf.zip"
shutil.make_archive(base_name="saved_model_tfidf", format="zip", root_dir=save_dir)
print(f"Model and vectorizer saved to {save_dir} and zipped as {zip_filename}")

if _IN_COLAB:
    print("Starting download of the zip (Colab)...")
    colab_files.download(zip_filename)

In [ ]:
# 1️⃣1️⃣ EVALUATION / TESTING
# =========================
print("\nStarting evaluation on test set...")

# Predict labels
y_pred = multi_clf.predict(X_test_tfidf)  # shape (n_samples, n_outputs)

# For probabilities per aspect (for mAP) — ensure estimator supports predict_proba
probas_per_aspect = []
for i, est in enumerate(multi_clf.estimators_):
    if hasattr(est, "predict_proba"):
        probas = est.predict_proba(X_test_tfidf)  # shape (n_samples, n_classes_for_aspect)
    else:
        # fallback: use decision_function and convert to pseudo-prob via softmax
        try:
            dec = est.decision_function(X_test_tfidf)
            # if binary, shape (n_samples,) -> expand
            if dec.ndim == 1:
                dec = np.vstack([-dec, dec]).T
            # softmax
            exp = np.exp(dec - np.max(dec, axis=1, keepdims=True))
            probas = exp / np.sum(exp, axis=1, keepdims=True)
        except Exception:
            # fall back to one-hot of predicted label
            preds = est.predict(X_test_tfidf)
            n_classes = len(np.unique(np.concatenate([y_train[:, i], y_test[:, i]])))
            probas = np.zeros((X_test_tfidf.shape[0], n_classes))
            for idx, p in enumerate(preds):
                # find index of class in est.classes_
                try:
                    class_idx = list(est.classes_).index(p)
                    probas[idx, class_idx] = 1.0
                except ValueError:
                    pass
    probas_per_aspect.append(probas)

# Compute metrics per aspect
metrics = []
for i, aspect in enumerate(aspect_names):
    y_true = y_test[:, i]
    y_pr = y_pred[:, i]
    y_prob = probas_per_aspect[i]  # shape (n_samples, n_classes_aspect)

    acc  = accuracy_score(y_true, y_pr)
    prec = precision_score(y_true, y_pr, average='macro', zero_division=0)
    rec  = recall_score(y_true, y_pr, average='macro', zero_division=0)
    f1   = f1_score(y_true, y_pr, average='macro', zero_division=0)

    # mAP: we need y_true in one-hot and y_prob aligned to same classes
    classes = multi_clf.estimators_[i].classes_
    # Binarize y_true to shape (n_samples, n_classes)
    y_true_bin = label_binarize(y_true, classes=classes)
    # If only one class in classes, average_precision_score will fail; handle it
    try:
        mAP = average_precision_score(y_true_bin, y_prob, average='macro')
    except Exception:
        mAP = 0.0

    metrics.append({
        "Aspect": aspect,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1,
        "mAP": mAP,
        "Classes": classes.tolist()
    })

# Save metrics to file and print
results_dir = "results"
os.makedirs(results_dir, exist_ok=True)
results_path = os.path.join(results_dir, "evaluation_results.txt")

with open(results_path, "w", encoding="utf-8") as f:
    f.write("📊 Evaluation Results\n")
    f.write("=" * 60 + "\n")
    for m in metrics:
        f.write(f"{m['Aspect']}\n")
        f.write(f"  Classes: {m['Classes']}\n")
        f.write(f"  Accuracy: {m['Accuracy']:.4f}\n")
        f.write(f"  Precision (macro): {m['Precision']:.4f}\n")
        f.write(f"  Recall (macro): {m['Recall']:.4f}\n")
        f.write(f"  F1 (macro): {m['F1']:.4f}\n")
        f.write(f"  mAP: {m['mAP']:.4f}\n")
        f.write("-" * 60 + "\n")

    mean_f1 = np.mean([m['F1'] for m in metrics])
    mean_map = np.mean([m['mAP'] for m in metrics])
    f.write(f"\nOverall Mean F1: {mean_f1:.4f}\n")
    f.write(f"Overall Mean mAP: {mean_map:.4f}\n")

print("Evaluation results written to:", results_path)
print("\nPer-aspect metrics:")
pprint(metrics)

if _IN_COLAB:
    colab_files.download(results_path)

# Also save a CSV summary
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv(os.path.join(results_dir, "evaluation_results.csv"), index=False)

# Save confusion matrices and classification reports for deeper inspection
for i, aspect in enumerate(aspect_names):
    cm = confusion_matrix(y_test[:, i], y_pred[:, i], labels=multi_clf.estimators_[i].classes_)
    cr = classification_report(y_test[:, i], y_pred[:, i], zero_division=0)
    np.savetxt(os.path.join(results_dir, f"confusion_matrix_{aspect}.csv"), cm, delimiter=",", fmt='%d')
    with open(os.path.join(results_dir, f"class_report_{aspect}.txt"), "w", encoding='utf-8') as f:
        f.write(cr)

# Zip results (optional)
shutil.make_archive("results", 'zip', results_dir)
if _IN_COLAB:
    colab_files.download("results.zip")


In [ ]:
# 1️⃣2️⃣ HOW TO USE THE MODEL LOCALLY (INFERENCE EXAMPLES)
# =========================
print("\n✅ To use the saved model locally (after download), reload as follows:")

usage_example = """
# Example usage after downloading saved_model_tfidf.zip and extracting:
import joblib
model = joblib.load("saved_model_tfidf/multioutput_model.joblib")
vectorizer = joblib.load("saved_model_tfidf/vectorizer.joblib")

def predict_review(review_text):
    # normalize like training
    review_text_norm = normalize_arabic(review_text)
    X = vectorizer.transform([review_text_norm])
    preds = model.predict(X)[0]  # array of 5 labels
    return dict(zip({0}, preds))

# Example:
print(predict_review("الأسعار مرتفعة لكن الأطباء ممتازون"))
""".format(aspect_names)

print(usage_example)
print("✅ Pipeline complete. Good luck testing on Colab! If you want, I can now:")
print("  • convert this to a ready-to-download .ipynb file,")
print("  • or swap LogisticRegression for RandomForest/SVC or add AraBERT hybrid features,")
print("  • or add cross-validation and hyperparameter search (GridSearchCV / RandomizedSearchCV).")
